# Уравнения газовой динамики (1D)

данный код решает 1D уравнения газовой динамики в декартовой системе координат. В качестве задачи можно переписать их под сферическую систему при наличии также силы тяжести.
Для верификации кода можно использовать статью [A. Mignone “High-order conservative reconstruction schemes for finite volume methods in cylindrical and spherical coordinates”, J Comput Phys 270:784–814 (2014)] https://doi.org/10.1016/j.jcp.2014.04.001, там же можно посмотреть детали

## 1. Декартова система координат (плоская геометрия)

Уравнения Эйлера в консервативной форме:

$$
\begin{cases}
\frac{\partial \rho}{\partial t} + \frac{\partial (\rho u)}{\partial x} = 0 & \text{(уравнение неразрывности)} \\
\frac{\partial (\rho u)}{\partial t} + \frac{\partial (\rho u^2 + p)}{\partial x} = 0 & \text{(уравнение импульса)} \\
\frac{\partial E}{\partial t} + \frac{\partial (u(E + p))}{\partial x} = 0 & \text{(уравнение энергии)}
\end{cases}
$$

где:
- $\rho$ - плотность газа
- $u$ - скорость
- $p$ - давление
- $E = \rho e + \frac{1}{2}\rho u^2$ - полная энергия на единицу объема
- $e$ - удельная внутренняя энергия

## 2. Сферическая система координат (радиальная симметрия)

Для сферически-симметричного случая уравнения имеют вид:

$$
\begin{cases}
\frac{\partial \rho}{\partial t} + \frac{1}{r^2}\frac{\partial (r^2 \rho u)}{\partial r} = 0 \\
\frac{\partial (\rho u)}{\partial t} + \frac{1}{r^2}\frac{\partial (r^2 (\rho u^2 + p))}{\partial r} = \frac{2 p}{r} -\rho \frac{\partial \Phi}{\partial r} \\
\frac{\partial E}{\partial t} + \frac{1}{r^2}\frac{\partial (r^2 u(E + p))}{\partial r} = -\rho u \frac{\partial \Phi}{\partial r},
\end{cases}
$$


где $\Phi(r)$ - гравитационный потенциал.

## Замыкание системы

Уравнения замыкаются уравнением состояния идеального газа:

$$
p = (\gamma - 1)\rho e
$$

где $\gamma$ - показатель адиабаты.

### Площади и объемы в сферической геометрии:
- **Грани ячейки** (сферические поверхности):
  $$ S_{f,i+1/2} = 4\pi r_{i+1/2}^2 $$
  $$ S_{f,i-1/2} = 4\pi r_{i-1/2}^2 $$

- **Объем ячейки** (сферический слой):
  $$ V_i = \frac{4}{3}\pi (r_{i+1/2}^3 - r_{i-1/2}^3) $$

- **Центр ячейки**:
  $$ cx_i = \frac{2}{3}\frac{r_{i+1/2}^3 - r_{i-1/2}^3}{r_{i+1/2}^2 - r_{i-1/2}^2} $$

Для самогравитирующего сферического газа:

$$
\frac{\partial \Phi}{\partial r} = \frac{G M(r)}{r^2}
$$

где:
- $G$ - гравитационная постоянная
- $M(r)$ - масса внутри радиуса r:

$$
M(r) = 4\pi \int_0^r \rho(r') r'^2 dr'
$$



In [ ]:
'''
зададим библиотеки
'''
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output  # добавляем импорт clear_output и display
import time
from collections import namedtuple

In [ ]:
'''
#для использования метода конечного объема зададим одномерную сетку на призме единичного сечения 
#     +-----+-----+-----+-----+-----+-----+-----+-----+
#    /  1  /     /     /     /     /     /     /     /|
#   /     /     /     /     /     /     /     /     / |
#  +-----+-----+-----+-----+-----+-----+-----+-----+  |
#  | 1   |     |     |     |     |     |     |     |  |
#  |     |     |     |     |     |     |     |     |  +
#  |     |     |     |     |     |     |     |     | /
#  |     |     |     |     |     |     |     |     |/
#  +--dx-+-----+-----+-----+-----+-----+-----+-----+
#
#  в качестве задания ее нужно будет переделать под сферическую симметрию с fS_r = 4pi*r_f^2, cVol = 4pi((r+)^3 - (r-)^3)/3    
'''

def grid_setup(xmin, xmax, Nx):
    #наша расчетная область - это призма с объемом x1*x2*x3 = (xmax - xmin)*1*1, которую мы "нарезаем" вдоль x1
    #количество фиктивных ("ghost") ячеек = 2
    Ngc = 2
    #разрешение сетки dx
    dx = (xmax - xmin)/Nx
    
    #задаем площадь грани: fSurf = 1*1
    fSx = np.full((Nx + 1), 1.0)

    #задаем объем ячейки: сVol = 1*1*dx
    cVol = np.full((Nx), 1.0 * dx)
    
    #координата центра ячейки
    cx = np.linspace(xmin - (2 - 0.5) * dx, xmax + (2 - 0.5) * dx, Nx + 2 * 2, dtype=np.double)
    
    #координата грани ячейки
    fx = np.linspace(xmin - 2 * dx, xmax + 2 * dx, Nx + 2 * 2 + 1, dtype=np.double)

    Grid = namedtuple('Grid', [
        'dx', 'fSx', 'cVol',
        'cx', 'fx', 'Nx', 'xmin', 'xmax'
    ])

    #возвращаем структуру данных про сетку
    return Grid(
        dx, fSx, cVol,
        cx, fx, Nx, xmin, xmax
    )
    

In [ ]:
#состояние среды  HS = hydro state 
def hydro_state(Nx):
    
    dens = np.ones(Nx + 4, dtype=np.double)
    pres = np.ones(Nx + 4, dtype=np.double)
    velx = np.zeros(Nx + 4, dtype=np.double)
    momx = np.zeros(Nx + 4, dtype=np.double)
    ener = np.ones(Nx + 4, dtype=np.double)

    
    HS = {
        'dens' : dens,
        'pres' : pres,
        'velx' : velx,
        'momx' : momx,
        'ener' : ener
    }

    return HS

In [ ]:
'''
ограничитель наклона для схем со вторым порядком точности 
обеспечивает монотонность решения вблизи разрывов без ухудшения его качества в областях гладкости
-- не вносит в решение нефизичного осцилляторного поведения
'''
def limiter(x, y):

    #smoothness analyzer 
    r = (y + 1e-14) / (x + 1e-14)
    
    #van Leer limiter  function
    xy = 1.0 * x * (np.abs(r) + r) / (1.0 + np.abs(r))
    
    return xy

'''
реконструкция переменных
'''
def PLM_rec(grid, var):

    #переобозначим для удобства
    Nxr = grid.Nx + 2

    '''градиенты'''
    #левая грань 
    grad_L = (var[1:Nxr] - var[0:Nxr-1]) / (grid.cx[1:Nxr] - grid.cx[0:Nxr-1])        
    #искомая грань
    grad_C = (var[2:Nxr+1] - var[1:Nxr]) / (grid.cx[2:Nxr+1] - grid.cx[1:Nxr])       
    #правая грань
    grad_R = (var[3:Nxr+2] - var[2:Nxr+1]) / (grid.cx[3:Nxr+2] - grid.cx[2:Nxr+1])
            
    #левое значение
    var_L = var[1:Nxr] + (grid.fx[2:-2] - grid.cx[1:-2]) * limiter(grad_L, grad_C)
    #правое значение
    var_R = var[2:Nxr+1] + (grid.fx[2:-2] - grid.cx[2:-1]) * limiter(grad_C, grad_R)

    return var_L, var_R
    

In [ ]:
'''решение задачи Римана'''
def Riemann_solver(rhol, rhor, vxl, vxr, pl, pr, gamma):
    
    #left conservative state
    mass_L = rhol
    momx_L = rhol * vxl
    etot_L = pl / (gamma - 1.0) + rhol * vxl * vxl / 2.0 
    
    #right conservative state
    mass_R = rhor
    momx_R = rhor * vxr
    etot_R = pr / (gamma - 1.0) + rhor * vxr * vxr / 2.0
    
    #left fluxes
    Fmass_L = rhol * vxl
    Fmomx_L = rhol * vxl * vxl + pl
    Fetot_L = vxl * (pl + etot_L)
    
    #right fluxes
    Fmass_R = rhor * vxr
    Fmomx_R = rhor * vxr * vxr + pr
    Fetot_R = vxr * (pr + etot_R)

    #left and right sound speeds 
    csl = np.sqrt(gamma * pl / rhol)
    csr = np.sqrt(gamma * pr / rhor)    
    
    #maximal and minimal eigenvalues estimate according to Davis (1988)
    Sl = np.minimum(vxl, vxr) - np.maximum(csl, csr)
    Sr = np.maximum(vxl, vxr) + np.maximum(csl, csr)
        
    #contact wave speed in HLLC approximation
    Sstar = (pr - pl + rhol * vxl * (Sl - vxl) - 
        rhor * vxr * (Sr - vxr)) / (rhol * (Sl - vxl) - rhor * (Sr - vxr))
        
    #conservative fluid state in the regions in both sides from the contact discontinuity
    #left starred state
    massS_L = rhol * (Sl - vxl) / (Sl - Sstar) 
    momxS_L = massS_L * Sstar 
    etotS_L = massS_L * ( etot_L / rhol + (Sstar - vxl) * (Sstar + pl / rhol / (Sl - vxl)) ) 
        
    #right starred state
    massS_R = rhor * (Sr - vxr) / (Sr - Sstar)
    momxS_R = massS_R * Sstar 
    etotS_R = massS_R * ( etot_R / rhor + (Sstar - vxr) * (Sstar + pr / rhor / (Sr - vxr)) ) 
        
    # calculation of the flux using HLLC approximate Riemann fan 
    # 4 states between left shock, contact wave, and right shock
    Fmass = np.where(Sl >= 0.0, Fmass_L, 
        np.where((Sl < 0.0) & (Sstar >= 0.0), Fmass_L + Sl * (massS_L - mass_L),
            np.where((Sstar < 0.0) & (Sr >= 0.0), Fmass_R + Sr * (massS_R - mass_R), Fmass_R)))
        
    Fmomx = np.where(Sl >= 0.0, Fmomx_L, 
        np.where((Sl < 0.0) & (Sstar >= 0.0), Fmomx_L + Sl * (momxS_L - momx_L),
            np.where((Sstar < 0.0) & (Sr >= 0.0), Fmomx_R + Sr * (momxS_R - momx_R), Fmomx_R)))

    Fetot = np.where(Sl >= 0.0, Fetot_L, 
        np.where((Sl < 0.0) & (Sstar >= 0.0), Fetot_L + Sl * (etotS_L - etot_L),
            np.where((Sstar < 0.0) & (Sr >= 0.0), Fetot_R + Sr * (etotS_R - etot_R), Fetot_R)))
        
    #return approximate Riemann flux for gas dynamics -- 
    #5 fluxes for conservative variables (mass, three components of momentum and energy)
    return Fmass, Fmomx, Fetot


In [ ]:
def gravity_acc(grid, HS):

    #ускорение определено только в реальных ячейках
    gravx = np.zeros(Nx, dtype=np.double)    

    return gravx

In [ ]:
'''
зададим решение в начальный момент времени 
'''
def hydro_init_cond(grid, HS):

    gamma = 7.0/5.0

    for i in range(2, grid.Nx + 2):
        if grid.cx[i] < 0.5:
            HS['dens'][i] = 1.0
            HS['pres'][i] = 1.0
            HS['velx'][i] = 0.0
        else:
            HS['dens'][i] = 0.125
            HS['pres'][i] = 0.1
            HS['velx'][i] = 0.0

    HS['momx'] = HS['dens']*HS['velx']
    HS['ener'] = HS['pres']/(gamma - 1.0) + HS['dens']*HS['velx']**2/2.0

    phys_time_fin = 0.2
    
    #возвращаем начальное условие
    return HS, gamma, phys_time_fin

In [ ]:
'''
#здесь начинается основной код программы
#-------------------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------
'''
#задаем нашу сетку - координаты границ и число ячеек
xmin = 0.0
xmax = 1.0
Nx = 200
grid = grid_setup(xmin,xmax,Nx)

#инициализируем массивы решения
HS0 = hydro_state(Nx)
HSh = hydro_state(Nx)
HS1 = hydro_state(Nx)

#зададим параметр Куранта-Фридрихса-Леви
CFL = 0.8

#начальное условие
HS0, gamma, phys_time_fin = hydro_init_cond(grid, HS0)

#задаем начальное время равным нулю
phys_time = 0.0
#задаем начальное число шагов
n_timestep = 0

#если не работает ipython - можно поставить runtimeplot = 0
runtimeplot = 1
if runtimeplot == 1:
    # Создание фигуры для графика
    fig, ax = plt.subplots()
    line, = ax.plot(grid.cx[2:-2], HS0['dens'][2:-2])
    ax.set_title('density at time = ' + str(np.round(phys_time, 3)))
    ax.set_xlabel('x')
    ax.set_ylabel('dens')
    plt.close()  # Закрываем фигуру, чтобы она не отображалась сразу


#цикл по времени
while phys_time < phys_time_fin - 1e-14:
    
    #задаем шаг по времени и обновляем счетчик
    dt = np.min((grid.fx[4:-1] - grid.fx[3:-2])/(np.abs(HS0['velx'][2:-2]) + np.sqrt(gamma*HS0['pres'][2:-2]/HS0['dens'][2:-2])))
    dt = min(CFL * dt, phys_time_fin - phys_time)
    phys_time += dt
    n_timestep += 1

    #считаем силу тяжести (пока не сделано)
    gravx = gravity_acc(grid, HS0)

    
    '''ШАГ ПРЕДИКТОРА'''
    #задаем граничные условия с использованием фиктивных ячеек (в данном примере у нас стенки)
    for i in range(0,2):
        #внутренняя граница
        HS0['dens'][i] = HS0['dens'][3-i]
        HS0['pres'][i] = HS0['pres'][3-i]
        HS0['velx'][i] = -HS0['velx'][3-i]
        #внешняя граница
        HS0['dens'][Nx+2+i] = HS0['dens'][Nx+1-i]
        HS0['pres'][Nx+2+i] = HS0['pres'][Nx+1-i]
        HS0['velx'][Nx+2+i] = -HS0['velx'][Nx+1-i]

    dens_L, dens_R = PLM_rec(grid, HS0['dens'])
    pres_L, pres_R = PLM_rec(grid, HS0['pres'])
    velx_L, velx_R = PLM_rec(grid, HS0['velx'])

    Fmass, Fmomx, Fener = Riemann_solver(dens_L, dens_R, velx_L, velx_R, pres_L, pres_R, gamma)
    
    #считаем массу, импульс и энергию
    HSh['dens'][2:-2] = HS0['dens'][2:-2] - dt*(Fmass[1:]*grid.fSx[1:] - Fmass[:-1]*grid.fSx[:-1])/grid.cVol
    HSh['momx'][2:-2] = HS0['momx'][2:-2] - dt*(Fmomx[1:]*grid.fSx[1:] - Fmomx[:-1]*grid.fSx[:-1])/grid.cVol
    HSh['ener'][2:-2] = HS0['ener'][2:-2] - dt*(Fener[1:]*grid.fSx[1:] - Fener[:-1]*grid.fSx[:-1])/grid.cVol
    
    #учет сил и источников (в случае сферической геометрии появятся члены с кривизной)
    HSh['momx'][2:-2] = HSh['momx'][2:-2] + dt*HS0['dens'][2:-2]*gravx
    HSh['ener'][2:-2] = HSh['ener'][2:-2] + dt*HS0['dens'][2:-2]*gravx*HS0['velx'][2:-2]
    
    #считаем скорость и давление
    HSh['velx'][2:-2] = HSh['momx'][2:-2]/HSh['dens'][2:-2]
    HSh['pres'][2:-2] = (gamma-1.0)*(HSh['ener'][2:-2] - HSh['dens'][2:-2]*HSh['velx'][2:-2]**2/2.0)
    
    '''ШАГ КОРРЕКТОРА'''
    #задаем граничные условия с использованием фиктивных ячеек (в данном примере у нас стенки)
    for i in range(0,2):
        #внутренняя граница
        HSh['dens'][i] = HSh['dens'][3-i]
        HSh['pres'][i] = HSh['pres'][3-i]
        HSh['velx'][i] = -HSh['velx'][3-i]
        #внешняя граница
        HSh['dens'][Nx+2+i] = HSh['dens'][Nx+1-i]
        HSh['pres'][Nx+2+i] = HSh['pres'][Nx+1-i]
        HSh['velx'][Nx+2+i] = -HSh['velx'][Nx+1-i]
    
    dens_L, dens_R = PLM_rec(grid, HSh['dens'])
    pres_L, pres_R = PLM_rec(grid, HSh['pres'])
    velx_L, velx_R = PLM_rec(grid, HSh['velx'])

    Fmass, Fmomx, Fener = Riemann_solver(dens_L, dens_R, velx_L, velx_R, pres_L, pres_R, gamma)

    #считаем массу, импульс и энергию
    HS1['dens'][2:-2] = HS0['dens'][2:-2]/2.0 + HSh['dens'][2:-2]/2.0 - dt*(Fmass[1:]*grid.fSx[1:] - Fmass[:-1]*grid.fSx[:-1])/grid.cVol/2.0
    HS1['momx'][2:-2] = HS0['momx'][2:-2]/2.0 + HSh['momx'][2:-2]/2.0 - dt*(Fmomx[1:]*grid.fSx[1:] - Fmomx[:-1]*grid.fSx[:-1])/grid.cVol/2.0
    HS1['ener'][2:-2] = HS0['ener'][2:-2]/2.0 + HSh['ener'][2:-2]/2.0 - dt*(Fener[1:]*grid.fSx[1:] - Fener[:-1]*grid.fSx[:-1])/grid.cVol/2.0

    #учет сил и источников (в случае сферической геометрии появятся члены с кривизной)
    HS1['momx'][2:-2] = HS1['momx'][2:-2] + (dt/2.0)*HSh['dens'][2:-2]*gravx
    HS1['ener'][2:-2] = HS1['ener'][2:-2] + (dt/2.0)*HSh['dens'][2:-2]*gravx*HSh['velx'][2:-2]
    
    #считаем скорость и давление
    HS1['velx'][2:-2] = HS1['momx'][2:-2]/HS1['dens'][2:-2]
    HS1['pres'][2:-2] = (gamma-1.0)*(HS1['ener'][2:-2] - HS1['dens'][2:-2]*HS1['velx'][2:-2]**2/2.0)

    #обновляем переменные 
    HS0['dens'] = HS1['dens']
    HS0['pres'] = HS1['pres']
    HS0['velx'] = HS1['velx']
    HS0['momx'] = HS1['momx']
    HS0['ener'] = HS1['ener']
    
    #вывод решения на экран
    if ((n_timestep%20 == 0 or phys_time > phys_time_fin - 1e-13) & (runtimeplot == 1)):
        print('num of timesteps = ', n_timestep)
        line.set_data(grid.cx[2:-2], HS0['dens'][2:-2])
        ax.set_title('sol at time = '+ str(np.round(phys_time, 2)))
        
        # Автоматическое масштабирование осей
        ax.relim()
        ax.autoscale_view()
        
        clear_output(wait=True)
        # Обновление графика с задержкой
        plt.pause(0.1)
        display(fig)


plt.ioff()
plt.show()

print('end of simulation')